In [1]:
%pip install -Uq adapters
%pip install -q datasets
%pip install -q accelerate

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
  from datasets import load_dataset

  dataset = load_dataset("super_glue", "cb")
  dataset.num_rows
  

e:\Research\cuda_env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


{'train': 250, 'validation': 56, 'test': 250}

In [3]:
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['premise', 'hypothesis', 'idx', 'label'],
        num_rows: 250
    })
    validation: Dataset({
        features: ['premise', 'hypothesis', 'idx', 'label'],
        num_rows: 56
    })
    test: Dataset({
        features: ['premise', 'hypothesis', 'idx', 'label'],
        num_rows: 250
    })
})


In [4]:
dataset["train"].features

{'premise': Value(dtype='string', id=None),
 'hypothesis': Value(dtype='string', id=None),
 'idx': Value(dtype='int32', id=None),
 'label': ClassLabel(names=['entailment', 'contradiction', 'neutral'], id=None)}

In [5]:
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")


def encode_batch(batch):
    """Encodes a batch of input data using the model tokenizer."""
    return tokenizer(
        batch["premise"],
        batch["hypothesis"],
        max_length=180,
        truncation=True,
        padding="max_length",
    )


# Encode the input data
dataset = dataset.map(encode_batch, batched=True)
# The transformers model expects the target class column to be named "labels"
dataset = dataset.rename_column("label", "labels")
# Transform to pytorch tensors and only output the required columns
dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])

In [6]:
from transformers import BertConfig
from adapters import BertAdapterModel

id2label = {
    id: label for (id, label) in enumerate(dataset["train"].features["labels"].names)
}

config = BertConfig.from_pretrained(
    "bert-base-uncased",
    id2label=id2label,
)
model = BertAdapterModel.from_pretrained(
    "bert-base-uncased",
    config=config,
)

Some weights of BertAdapterModel were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['heads.default.3.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [7]:
%pip install adapter-transformers

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [8]:
from adapters.composition import Fuse

# Load the pre-trained adapters we want to fuse
model.load_adapter("nli/multinli@ukp", load_as="multinli", with_head=False)
model.load_adapter("sts/qqp@ukp", with_head=False)
model.load_adapter("nli/qnli@ukp", with_head=False)
# Add a fusion layer for all loaded adapters
adapter_setup = Fuse("multinli", "qqp", "qnli")
model.add_adapter_fusion(adapter_setup)

# Add a classification head for our target task
model.add_classification_head("cb", num_labels=len(id2label))

Automatic redirect to HF Model Hub repo 'AdapterHub/bert-base-uncased_nli_multinli_pfeiffer'. Please switch to the new ID to remove this warning.
Fetching 7 files: 100%|██████████| 7/7 [00:00<?, ?it/s]
Automatic redirect to HF Model Hub repo 'AdapterHub/bert-base-uncased_sts_qqp_pfeiffer'. Please switch to the new ID to remove this warning.
Fetching 7 files: 100%|██████████| 7/7 [00:00<?, ?it/s]
Automatic redirect to HF Model Hub repo 'AdapterHub/bert-base-uncased_nli_qnli_pfeiffer'. Please switch to the new ID to remove this warning.
Fetching 7 files: 100%|██████████| 7/7 [00:00<?, ?it/s]


In [9]:
# Unfreeze and activate fusion setup
model.train_adapter_fusion(adapter_setup)

There are adapters available but none are activated for the forward pass.


In [10]:
import numpy as np
from transformers import TrainingArguments, EvalPrediction
from adapters import AdapterTrainer

training_args = TrainingArguments(
    learning_rate=5e-5,
    num_train_epochs=5,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    logging_steps=200,
    output_dir="./training_output",
    overwrite_output_dir=True,
    # The next line is important to ensure the dataset labels are properly passed to the model
    remove_unused_columns=False,
)


def compute_accuracy(p: EvalPrediction):
    preds = np.argmax(p.predictions, axis=1)
    return {"acc": (preds == p.label_ids).mean()}


trainer = AdapterTrainer(
    model=model,
    args=training_args,
    train_dataset=dataset["train"],
    eval_dataset=dataset["validation"],
    compute_metrics=compute_accuracy,
)

In [11]:
trainer.train()

Step,Training Loss


TrainOutput(global_step=40, training_loss=0.7753856658935547, metrics={'train_runtime': 717.9024, 'train_samples_per_second': 1.741, 'train_steps_per_second': 0.056, 'total_flos': 149577058350000.0, 'train_loss': 0.7753856658935547, 'epoch': 5.0})